## Importar librerias

In [2]:
# Manejo de datos
import numpy as np
import pandas as pd

## Lectura Dataset

In [ ]:
# Cargar el archivo Excel
df = pd.read_excel("accesos.xlsx")

# Leer días de vacaciones desde un archivo txt
with open('dias_vacaciones.txt', 'r') as f:
    dias_vacaciones = [pd.to_datetime(line.strip(), errors='coerce') for line in f if line.strip()]

# Asegurar que la fecha esté en formato datetime
df['Fecha Completa'] = pd.to_datetime(df['Fecha Completa'])

# Crear variables temporales relevantes
df['Día'] = df['Fecha Completa'].dt.day
df['Mes'] = df['Fecha Completa'].dt.month
df['Año'] = df['Fecha Completa'].dt.year

# Codificar Jornada, Pregrado y Postgrado
df['Jornada'] = df['Jornada'].astype('category').cat.codes
df['Pregrado'] = df['Pregrado'].map({'SI': 1, 'NO': 0})
df['Postgrado'] = df['Postgrado'].map({'SI': 1, 'NO': 0})

# Crear una clave de fecha sin hora
df['Fecha'] = df['Fecha Completa'].dt.date

# Crear columna exógena de vacaciones
df['Vacaciones'] = df['Fecha Completa'].dt.normalize().isin(dias_vacaciones).astype(int)

# Agrupar y contar accesos
daily_counts = df.groupby(['Fecha', 'Día', 'Mes', 'Año']).agg({
    'Pregrado': 'sum',
    'Postgrado': 'sum',
    'Fecha Completa': 'count',  # esto cuenta los accesos
    'Vacaciones': 'max'  # max para saber si es día de vacaciones
}).rename(columns={'Fecha Completa': 'Accesos'}).reset_index()

# Crear rango completo de fechas
fecha_inicio = df['Fecha'].min()
fecha_fin = df['Fecha'].max()
fechas_completas = pd.DataFrame({'Fecha': pd.date_range(fecha_inicio, fecha_fin)})

# Extraer día, mes y año para el rango completo
fechas_completas['Día'] = fechas_completas['Fecha'].dt.day
fechas_completas['Mes'] = fechas_completas['Fecha'].dt.month
fechas_completas['Año'] = fechas_completas['Fecha'].dt.year
fechas_completas['Fecha'] = fechas_completas['Fecha'].dt.date  # convertir a date para coincidir

# Marcar vacaciones en el rango completo
fechas_completas['Vacaciones'] = pd.to_datetime(fechas_completas['Fecha']).isin(dias_vacaciones).astype(int)

# Unir con daily_counts
daily_counts_full = fechas_completas.merge(
    daily_counts.drop(columns=['Vacaciones']),  # quitamos la columna para evitar duplicados
    on=['Fecha', 'Día', 'Mes', 'Año'],
    how='left'
)

# Rellenar NaN con 0 en las columnas numéricas
for col in ['Pregrado', 'Postgrado', 'Accesos']:
    daily_counts_full[col] = daily_counts_full[col].fillna(0).astype(int)


KeyboardInterrupt: 

In [17]:
daily_counts_full

,Fecha,Día,Mes,Año,Vacaciones,Pregrado,Postgrado,Accesos
0,2023-12-14,14,12,2023,0,665,17,953
1,2023-12-15,15,12,2023,0,395,5,549
2,2023-12-16,16,12,2023,0,48,2,71
3,2023-12-17,17,12,2023,0,0,0,0
4,2023-12-18,18,12,2023,0,500,15,739
...,...,...,...,...,...,...,...,...
506,2025-05-03,3,5,2025,0,0,0,0
507,2025-05-04,4,5,2025,0,0,0,0
508,2025-05-05,5,5,2025,0,2688,15,2750
509,2025-05-06,6,5,2025,0,1808,5,1866


In [18]:
daily_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Fecha       334 non-null    object 
 1   Día         334 non-null    int32  
 2   Mes         334 non-null    int32  
 3   Año         334 non-null    int32  
 4   Pregrado    334 non-null    float64
 5   Postgrado   334 non-null    float64
 6   Accesos     334 non-null    int64  
 7   Vacaciones  334 non-null    int32  
dtypes: float64(2), int32(4), int64(1), object(1)
memory usage: 15.8+ KB
